- https://github.com/SKT-AI/KoGPT2
- https://wikidocs.net/157896

# Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

default_path = '/content/drive/My Drive'

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

# Data Load

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
dir = Path("/content/drive/MyDrive/메타버스 아카데미/본과정/Python")

In [ ]:
data1 = pd.read_csv(str(dir.joinpath("AllData.csv")))
data1

,req,res
0,너 좋아하는 차 종류 있어?,무슨 차? 자동차? 마시는 차?
1,ㅋㅋ 마시는 차 말한 거야!,"아하 나 둥글레, 옥수수, 보리차 좋아해"
2,완전 곡물류 좋아하네 ㅋㅋ,야쓰 끓이기 귀찮아서 냉침해 먹어
3,그럼 오래 걸리지 않아?,끓이는 것보다는 훨씬 오래 걸리지 ㅠ
4,근데 냉침 하는 것도 귀찮겠다 ㅜㅠ,응! 그래서 매일은 안 먹고 가끔 마셔
...,...,...
88969,아르바이트 면접 언제야?,내일 아르바이트 면접이야!
88970,무슨 아르바이트 지원했어?,나 카페 아르바이트 지원했어
88971,카페는 경력자 원하지 않아?,맞아 나 전에 카페 경력 일년 정도 있어서 괜찮을걸?
88972,아 맞네 ㅋㅋ 너 카페 일했었네,응 근데 까먹었긴 한데 일 배우면 또 다해


In [ ]:
data1.rename(columns={"req":"Q", "res":"A"}, inplace=True)
data1["label"] = 0

In [ ]:
data2 = pd.read_csv(dir.joinpath("ChatbotData.csv"))
data2.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [ ]:
train_data = pd.concat([data2, data1], axis=0)
train_data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
88969,아르바이트 면접 언제야?,내일 아르바이트 면접이야!,0
88970,무슨 아르바이트 지원했어?,나 카페 아르바이트 지원했어,0
88971,카페는 경력자 원하지 않아?,맞아 나 전에 카페 경력 일년 정도 있어서 괜찮을걸?,0
88972,아 맞네 ㅋㅋ 너 카페 일했었네,응 근데 까먹었긴 한데 일 배우면 또 다해,0


In [ ]:
train_data.reset_index(drop=True, inplace=True)

# Preprocessing

In [ ]:
train_data["Q"] = train_data["Q"].str.split("/").str[0]
train_data["A"] = train_data["A"].str.split("/").str[0]

In [ ]:
train_data.shape

(100797, 3)

In [ ]:
train_data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
100792,아르바이트 면접 언제야?,내일 아르바이트 면접이야!,0
100793,무슨 아르바이트 지원했어?,나 카페 아르바이트 지원했어,0
100794,카페는 경력자 원하지 않아?,맞아 나 전에 카페 경력 일년 정도 있어서 괜찮을걸?,0
100795,아 맞네 ㅋㅋ 너 카페 일했었네,응 근데 까먹었긴 한데 일 배우면 또 다해,0


### ? ! . 양옆에 공백을 추가

In [ ]:
train_data = train_data.replace(regex=r'([?.!,])', value=r" \1 ")
train_data

,Q,A,label
0,12시 땡 !,하루가 또 가네요 .,0
1,1지망 학교 떨어졌어,위로해 드립니다 .,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠 .,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠 .,0
4,PPL 심하네,눈살이 찌푸려지죠 .,0
...,...,...,...
100792,아르바이트 면접 언제야 ?,내일 아르바이트 면접이야 !,0
100793,무슨 아르바이트 지원했어 ?,나 카페 아르바이트 지원했어,0
100794,카페는 경력자 원하지 않아 ?,맞아 나 전에 카페 경력 일년 정도 있어서 괜찮을걸 ?,0
100795,아 맞네 ㅋㅋ 너 카페 일했었네,응 근데 까먹었긴 한데 일 배우면 또 다해,0


In [ ]:
questions = train_data["Q"].to_list()
answers = train_data["A"].to_list()

데이터를 list에 저장한다. !?. 와 같은 기호들은 공백을 추가해 준다.

In [ ]:
print(questions[:5])
print(answers[:5])

['12시 땡 ! ', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네']
['하루가 또 가네요 . ', '위로해 드립니다 . ', '여행은 언제나 좋죠 . ', '여행은 언제나 좋죠 . ', '눈살이 찌푸려지죠 . ']


## 한글 맞춤법 검사

### stripping illegal characters out of xml

In [ ]:
# !git clone https://github.com/ssut/py-hanspell.git

Cloning into 'py-hanspell'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 110 (delta 16), reused 9 (delta 9), pack-reused 88
Receiving objects: 100% (110/110), 22.30 KiB | 11.15 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [ ]:
# %cd py-hanspell
# !python setup.py install

[Errno 2] No such file or directory: 'py-hanspell'
/content/py-hanspell
/usr/local/lib/python3.10/dist-packages/setuptools/dist.py:755: SetuptoolsDeprecationWarning: Invalid dash-separated options
!!

        ********************************************************************************
        Usage of dash-separated 'description-file' will not be supported in future
        versions. Please use the underscore name 'description_file' instead.

        By 2023-Sep-26, you need to update your project and remove deprecated calls
        or your builds will no longer be supported.

        See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
        ********************************************************************************

!!
  opt = self.warn_dash_deprecation(opt, section)
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ***********

In [ ]:
# from hanspell import spell_checker

In [ ]:
# spell_checker.check(u'안녕 하세요. 저는 한국인 입니다. 이문장은 한글로 작성됬습니다.').checked

'안녕하세요. 저는 한국인입니다. 이 문장은 한글로 작성됐습니다.'

In [ ]:
# train_data = train_data.replace(regex=r"[<&>'\"]", value=" ")

In [ ]:
# train_data["Q"] = train_data["Q"].apply(lambda text: spell_checker.check(str(text)).checked)
# train_data["A"] = train_data["A"].apply(lambda text: spell_checker.check(str(text)).checked)
# train_data

KeyboardInterrupt: ignored

# KoGPT2

참고:
- https://wikidocs.net/157896

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.1 MB/s eta 0:00:00


In [ ]:
!pip install git+https://github.com/PyTorchLightning/pytorch-lightning

  Cloning https://github.com/PyTorchLightning/pytorch-lightning to /tmp/pip-req-build-3vic_5kj
  Running command git clone --filter=blob:none --quiet https://github.com/PyTorchLightning/pytorch-lightning /tmp/pip-req-build-3vic_5kj
  Resolved https://github.com/PyTorchLightning/pytorch-lightning to commit 2af425328a64faac492d4c248b55d799668aa79c
  Running command git submodule update --init --recursive -q
  Encountered 31 file(s) that should have been pointers, but weren't:
        .notebooks/course_UvA-DL/01-introduction-to-pytorch.ipynb
        .notebooks/course_UvA-DL/02-activation-functions.ipynb
        .notebooks/course_UvA-DL/03-initialization-and-optimization.ipynb
        .notebooks/course_UvA-DL/04-inception-resnet-densenet.ipynb
        .notebooks/course_UvA-DL/05-transformers-and-MH-attention.ipynb
        .notebooks/course_UvA-DL/06-graph-neural-networks.ipynb
        .notebooks/course_UvA-DL/07-deep-energy-based-generative-models.ipynb
        .notebooks/course_UvA-DL/08-

# Import

In [ ]:
import math
import numpy as np
import pandas as pd
import random
import re
import torch
import urllib.request
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning import LightningModule
from pytorch_lightning.callbacks import ModelCheckpoint
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup


# Tokens

In [ ]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'


# Tokenizer

In [ ]:
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK)
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


# Dataset

In [ ]:
# 챗봇 데이터를 처리하는 클래스를 만든다.
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=40):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", str(q))  # 구둣점들을 제거한다.

        a = turn["A"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", str(a))  # 구둣점들을 제거한다.

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)


# Collate Batch

In [ ]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)


# Hyper Parameter

In [ ]:
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epochs = 10
Sneg = -1e18
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Initialize Dataset, DataLoader

In [ ]:
train_set = ChatbotDataset(train_data, max_len=40)

#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=2, shuffle=True, collate_fn=collate_batch,)


# Train

In [ ]:
model.to(device)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [ ]:
import wandb

wandb.login()
wandb.init(project="kogpt2")
wandb.config = {
    "epochs": epochs,
    "batch_size": train_dataloader.batch_size,
    "learning_rate": optimizer.param_groups[0]['lr'],
}
wandb.watch(model)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: blessian. Use `wandb login --relogin` to force relogin


[]

In [ ]:
model.to(device)
best_loss = float("inf")
print ("start")

for i in range(epochs):
    batchloss = 0.0
    progress = tqdm(train_dataloader)

    for batch_idx, samples in enumerate(progress):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        token_ids, mask, label = token_ids.to(device), mask.to(device), label.to(device)
        out = model(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        batchloss += avg_loss
        avg_loss.backward()
        # 학습 끝
        optimizer.step()

    print("epoch:", i+1, "|", "loss:", batchloss.to(device).item()/len(train_dataloader))

    if (i+1) % 10 == 0: torch.save(model.state_dict(), f"chatbot_gpt2_{i+1}.pth")
    if best_loss > batchloss.to(device).item():
        best_loss = batchloss.to(device).item()
        torch.save(model.state_dict(), f"chatbot_gpt2_best.pth")

    wandb.log({
        "Epoch": i+1,
        "Learning Rate": optimizer.param_groups[0]['lr'],
        "Loss": round(batchloss.to(device).item()/len(train_dataloader), 3),
    })
print ("end")


start


100%|██████████| 3150/3150 [21:20<00:00,  2.46it/s]


epoch: 1 | loss: 24.369434523809524


100%|██████████| 3150/3150 [21:22<00:00,  2.46it/s]


epoch: 2 | loss: 24.13044890873016


  9%|▉         | 285/3150 [01:57<19:36,  2.43it/s]


KeyboardInterrupt: ignored

In [ ]:
torch.save(model.state_dict(), "chatbot_gpt2_last.pth")

# Test

In [ ]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
model.load_state_dict(torch.load("chatbot_gpt2_best.pth"))
model.to("cpu")

sent = "0"

with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + sent + A_TKN + a)).unsqueeze(dim=0)
            pred = model(input_ids)
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))


user > 안녕
Chatbot > 안녕하세요
user > 나 집에 가고 싶어
Chatbot > 오오오오오오오옹
user > 오오오오옹?
Chatbot > ᄏᄏ 근데 그거 진짜 재밌어 ᄏᄏ
user > 저녁 뭐 먹을까?
Chatbot > 오늘은 점심 먹고 오세요
user > 점심 뭐 먹을까?
Chatbot > 오늘 점심 뭐 먹었어
user > 졸려
Chatbot > 졸려요
user > 너무 졸리다
Chatbot > 졸린가봐요
user > quit
